# Scalable model training pipeline for Stable Diffusion

<img src="https://anyscale-materials.s3.us-west-2.amazonaws.com/stable-diffusion/training_architecture_v3.jpeg" width="700px">

The preceding architecture diagram illustrates the model training process for Stable Diffusion. 

The training stage primarily consists of three steps:

1. **Load the preprocessed data**: Load preprocessed data to feed the model.
2. **Build a Stable Diffusion model**: Use a network architecture called a U-Net to build the model.
3. **Run the scalable training procedure**: Scale the base model from the previous step with Ray Train to enable running on a GPU compute cluster. 
	  
This notebook executes a fully self-contained module, `train.py`, that processes a small subset of the full 2 billion dataset on a small U-Net model to demonstrate the workload. You can parameterize the same module code to train over the full dataset. The `Running production-scale model training` section below summarizes the necessary changes for training over the full dataset. 

Run the following cell to perform the model training. The script loads the data, builds the model, and runs Ray to train and checkpoint the model. After the cell executes, view the generated model checkpoint files.

In [ ]:
!python scripts/train.py

## Running production-scale model training

If you're looking to scale your Stable Diffusion pre-training and potenitally use your own custom data, we're here to help 🙌 !

👉 **[Check out this link](https://forms.gle/9aDkqAqobBctxxMa8) so we can assist you**.


In case you would like to get an idea of the changes needed to scale the `train.py` script to the full dataset, below is a table that provides approximate guidance on the changes you need to make:

| Step | Change | Action |
| --- | --- | --- |
| 1 | Change the processed training and validation data paths | Point to the full dataset |
| 2 | Change the number of data loading workers | Change from 1 to 64 CPUs to load the data |
| 2 | Change the number of training workers | Change from 1 A10G GPU to 32 A100-80 GB GPUs to run the training |
| 4 | Change the batch size | Use 128 for a resolution of 256x256 images and 32 for a resolution of 512x512 images |
| 5 | Update the model config | Use the full U-Net model |
| 6 | Set `FSDP` as the distributed training strategy | Configure it to run in `SHARD_GRAD_OP` mode |
| 7 | Change the output path | Change to the desired remote storage output path  |
| 8 | Run the process script as an Anyscale Job | Submit a job instead of running in a notebook |
| 9 | Run the first phase of training | Use resolution 256x256 for a total of 550,000 steps |
| 10 | Run the second phase of training | Use resolution 512x512 for a total of 850,000 steps loading the checkpoint from the first phase |


In terms of infrastructure, you would provision 4 `p4de.24xlarge` instances for the entire process or use Anyscale's autoscaling capabilities to scale up and down as needed.